In [5]:
import sys
sys.path.append("../")

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import copy
from neural_net.mlp import MLP
from neural_net.nn_utils import compute_jacobian
from models.hd_regression_model import HDRegressionModel
from models.ld_regression_model import LDRegressionModel
from sampling.least_squares_sampler import LeastSquaresSampler
from sampling.sghmc_sampling import SGHMCsampler
from sampling.rate_schedulers import *
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from models.model_generators import *
from sampling.rate_schedulers import *
from neural_net.nn_utils import *

In [7]:
print("INFO: Training Network")
net = get_trained_network('../configs/nn_config.ini')
print("INFO: Generating Linear Model")
model, ld_model = generate_nn_linear_models('../configs/nn_config.ini', net)

INFO: Training Network
Epoch 0, Loss: 0.9714, Validation Loss: 3870.7981
Epoch 100, Loss: 0.8509, Validation Loss: 3387.1194
Epoch 200, Loss: 0.7614, Validation Loss: 2956.7720
Epoch 300, Loss: 0.6702, Validation Loss: 2601.6462
Epoch 400, Loss: 0.6054, Validation Loss: 2321.8655
Epoch 500, Loss: 0.5611, Validation Loss: 2109.4800
Epoch 600, Loss: 0.5244, Validation Loss: 1954.2017
Epoch 700, Loss: 0.4988, Validation Loss: 1843.7627
Epoch 800, Loss: 0.4775, Validation Loss: 1765.3984
Epoch 900, Loss: 0.4651, Validation Loss: 1707.8715
Epoch 1000, Loss: 0.4607, Validation Loss: 1663.5486
Epoch 1100, Loss: 0.4418, Validation Loss: 1625.6606
Epoch 1200, Loss: 0.4372, Validation Loss: 1593.6300
Epoch 1300, Loss: 0.4282, Validation Loss: 1565.1472
Epoch 1400, Loss: 0.4238, Validation Loss: 1538.8008
Epoch 1500, Loss: 0.4120, Validation Loss: 1514.9304
Epoch 1600, Loss: 0.4126, Validation Loss: 1492.8330
Epoch 1700, Loss: 0.4026, Validation Loss: 1472.6213
Epoch 1800, Loss: 0.3982, Validatio

In [8]:
d_dash = 501

In [9]:
sghmc_sampler = SGHMCsampler(model, 7500, ld_model.mean, 750, ld_model.mean, Minv=np.diag(np.full(d_dash, 1)), gamma=0.003, epsilon=0.001)

In [10]:
N = 10
sghmc_samples = np.zeros((N, d_dash))
for i in range(N):
	sghmc_samples[i] = sghmc_sampler.sample()

100%|██████████| 750/750 [01:10<00:00, 10.58it/s]


In [1]:
test_samples = np.random.multivariate_normal(np.zeros(d_dash), ld_model.Hinv, size=500)
fig, ax = plt.subplots(3,3)
for i in range(3):
    for j in range(3):
        a = 1
        b = 1
        while a == b:    
            a = np.random.randint(0, d_dash)
            b = np.random.randint(0, d_dash)
            if a != b: 
                ax[i][j].scatter(sghmc_samples[:,a], sghmc_samples[:,b], color='Black', marker='+', label="Least Squares Samples")
                ax[i][j].scatter(test_samples[:,a], test_samples[:,b], c='Grey', zorder=-1, label="Test Samples")
                ax[i][j].set_xlabel("x{}".format(a))
                ax[i][j].set_ylabel("x{}".format(b))
handles, labels = ax[1][2].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=1.1,
                    top=0.85,
                    wspace=0.4,
                    hspace=0.4)

NameError: name 'np' is not defined